# HyperCLOVAX 모델 배치 크기(Batch Size) 비교 분석

이 노트북은 원본 모델과 서로 다른 배치 크기로 미세조정된 HyperCLOVAX 모델의 성능을 비교합니다.

## 분석 목표
- 세 모델 간 정량적 성능 비교 (BLEU, ROUGE, 문자 정확도)
- 정성적 분석 (실제 출력 예시 비교)
- 카테고리별 성능 분석
- 추론 시간 및 효율성 비교
- 배치 크기 조정 효과 분석
- 결과 시각화

## 모델 정보
- **원본 모델**: `naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B`
- **배치 크기 1 모델**: `hyperclova-deobfuscation-lora-with-1-batch-size`
- **배치 크기 2 모델**: `hyperclova-deobfuscation-lora-with-2-batch-size`
- **배치 크기 4 모델**: `hyperclova-deobfuscation-lora-with-4-batch-size`
- **테스트 데이터**: `testdata.csv` (1,002 샘플)

## 1. 환경 설정 및 라이브러리 설치

In [ ]:
# GPU 확인
!nvidia-smi

# 필수 패키지 설치
!pip install -q transformers
!pip install -q peft
!pip install -q torch
!pip install -q datasets
!pip install -q evaluate
!pip install -q rouge-score
!pip install -q sacrebleu
!pip install -q sentencepiece
!pip install -q protobuf
!pip install -q matplotlib
!pip install -q seaborn
!pip install -q plotly
!pip install -q pandas
!pip install -q numpy
!pip install -q scikit-learn
!pip install -q tqdm

print("패키지 설치 완료")

In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from evaluate import load
from tqdm import tqdm
import time
import warnings
import os
warnings.filterwarnings('ignore')

# matplotlib 및 seaborn 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
sns.set_palette("husl")

# 이미지 저장 폴더 생성
image_save_dir = '/content/drive/MyDrive/Colab Notebooks/analysis_images'
os.makedirs(image_save_dir, exist_ok=True)
print(f"이미지 저장 폴더 생성: {image_save_dir}")

# 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 장치: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU 메모리: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

## 2. 데이터 로딩 및 전처리

In [ ]:
# Google Drive 연결 (Colab에서 실행 시)
try:
    from google.colab import drive
    import shutil
    
    # 기존 마운트 포인트가 있으면 정리
    mount_point = '/content/drive'
    if os.path.exists(mount_point):
        try:
            # 마운트 해제 시도
            print("기존 마운트 포인트 정리 중...")
            os.system(f'fusermount -u {mount_point} 2>/dev/null || true')
            shutil.rmtree(mount_point, ignore_errors=True)
        except:
            pass
    
    # Google Drive 마운트
    drive.mount(mount_point, force_remount=True)
    
    # 경로 설정
    BASE_PATH = '/content/drive/MyDrive/'
    MODEL_BATCH_1_PATH = BASE_PATH + 'hyperclova-deobfuscation-lora-with-1-batch-size'
    MODEL_BATCH_2_PATH = BASE_PATH + 'hyperclova-deobfuscation-lora-with-2-batch-size'
    MODEL_BATCH_4_PATH = BASE_PATH + 'hyperclova-deobfuscation-lora-with-4-batch-size'
    TEST_DATA_PATH = BASE_PATH + 'testdata.csv'
    
    # Google Drive 루트에 전용 분석 결과 폴더 생성
    analysis_root_dir = os.path.join(BASE_PATH, 'HyperCLOVAX_BatchSize_Analysis_Results')
    os.makedirs(analysis_root_dir, exist_ok=True)
    print(f"분석 결과 루트 폴더 생성: {analysis_root_dir}")
    
except ImportError:
    # 로컬 실행 시
    BASE_PATH = './'
    MODEL_BATCH_1_PATH = './hyperclova-deobfuscation-lora-with-1-batch-size'
    MODEL_BATCH_2_PATH = './hyperclova-deobfuscation-lora-with-2-batch-size'
    MODEL_BATCH_4_PATH = './hyperclova-deobfuscation-lora-with-4-batch-size'
    TEST_DATA_PATH = './testdata.csv'
    
    # 로컬용 분석 결과 폴더
    analysis_root_dir = './HyperCLOVAX_BatchSize_Analysis_Results'
    os.makedirs(analysis_root_dir, exist_ok=True)
    print(f"분석 결과 루트 폴더 생성: {analysis_root_dir}")

# 이미지 저장 폴더 생성 (분석 결과 폴더 내에)
image_save_dir = os.path.join(analysis_root_dir, 'visualization_images')
os.makedirs(image_save_dir, exist_ok=True)
print(f"이미지 저장 폴더 생성: {image_save_dir}")

print(f"\n경로 설정 완료:")
print(f"배치 크기 1 모델 경로: {MODEL_BATCH_1_PATH}")
print(f"배치 크기 2 모델 경로: {MODEL_BATCH_2_PATH}")
print(f"배치 크기 4 모델 경로: {MODEL_BATCH_4_PATH}")
print(f"테스트 데이터 경로: {TEST_DATA_PATH}")
print(f"분석 결과 저장 경로: {analysis_root_dir}")
print(f"이미지 저장 경로: {image_save_dir}")

In [ ]:
# 테스트 데이터 로드
test_df = pd.read_csv(TEST_DATA_PATH)
print(f"테스트 데이터 크기: {len(test_df)} 샘플")
print(f"컬럼 목록: {test_df.columns.tolist()}")
print("\n첫 5개 샘플:")
print(test_df.head())

# 데이터 통계
print("\n데이터 통계:")
print(f"- 총 샘플 수: {len(test_df)}")
print(f"- 원본 텍스트 평균 길이: {test_df['original'].str.len().mean():.1f}")
print(f"- 난독화 텍스트 평균 길이: {test_df['obfuscated'].str.len().mean():.1f}")

## 3. 모델 로딩

In [ ]:
# 베이스 모델 이름 설정
BASE_MODEL_NAME = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"

# 토크나이저 로드 (배치 크기 1 모델에서)
print("토크나이저 로딩 중...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_BATCH_1_PATH)
print(f"토크나이저 어휘 크기: {len(tokenizer)}")

In [ ]:
def load_model(model_path, model_name):
    """로라 모델을 로드합니다"""
    print(f"\n{model_name} 모델 로딩 중...")
    
    # 베이스 모델 로드
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    # LoRA 어댑터 적용
    model = PeftModel.from_pretrained(base_model, model_path)
    model = model.to(device)
    
    print(f"{model_name} 모델 로딩 완료")
    return model

def load_base_model():
    """원본 베이스 모델을 로드합니다"""
    print("\n원본 모델 로딩 중...")
    
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    base_model = base_model.to(device)
    
    print("원본 모델 로딩 완료")
    return base_model

# 모델 로드
base_model = load_base_model()
model_batch_1 = load_model(MODEL_BATCH_1_PATH, "배치 크기 1 모델")
model_batch_2 = load_model(MODEL_BATCH_2_PATH, "배치 크기 2 모델")
model_batch_4 = load_model(MODEL_BATCH_4_PATH, "배치 크기 4 모델")

print("모든 모델 로딩 완료!")

## 4. 추론 함수 정의

In [ ]:
def generate_deobfuscated_text(model, obfuscated_text, max_length=256):
    """난독화된 텍스트를 입력받아 원본 텍스트 생성"""
    prompt = f"""### 지시사항:
다음 난독화된 한국어 텍스트를 원래 텍스트로 복원해주세요.

난독화된 텍스트: {obfuscated_text}

### 응답:
"""
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    start_time = time.time()
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    inference_time = time.time() - start_time
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 응답 부분만 추출
    if "### 응답:" in response:
        response = response.split("### 응답:")[1].strip()
        # 불필요한 부분 제거
        if "<|endoftext|>" in response:
            response = response.split("<|endoftext|>")[0].strip()
    
    return response, inference_time

print("추론 함수 정의 완료")

## 5. 성능 평가 메트릭 정의

In [ ]:
# 평가 메트릭 로드
bleu = load("bleu")
rouge = load("rouge")

def calculate_character_accuracy(pred, ref):
    """문자 단위 정확도 계산"""
    if len(ref) == 0:
        return 1.0 if len(pred) == 0 else 0.0
    
    # 정확히 일치하는 문자 수 계산
    matches = sum(1 for i, char in enumerate(pred) if i < len(ref) and char == ref[i])
    return matches / len(ref)

def calculate_exact_match(pred, ref):
    """완전 일치 여부"""
    return 1.0 if pred.strip() == ref.strip() else 0.0

def calculate_metrics(predictions, references):
    """모든 메트릭 계산"""
    # BLEU 계산
    try:
        bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])['bleu']
    except:
        bleu_score = 0.0
    
    # ROUGE 계산
    try:
        rouge_scores = rouge.compute(predictions=predictions, references=references)
    except:
        rouge_scores = {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}
    
    # 문자 정확도 계산
    char_accuracies = [calculate_character_accuracy(pred, ref) for pred, ref in zip(predictions, references)]
    avg_char_accuracy = np.mean(char_accuracies)
    
    # 완전 일치율 계산
    exact_matches = [calculate_exact_match(pred, ref) for pred, ref in zip(predictions, references)]
    exact_match_rate = np.mean(exact_matches)
    
    return {
        'bleu': bleu_score,
        'rouge1': rouge_scores['rouge1'],
        'rouge2': rouge_scores['rouge2'],
        'rougeL': rouge_scores['rougeL'],
        'char_accuracy': avg_char_accuracy,
        'exact_match': exact_match_rate,
        'char_accuracies': char_accuracies,
        'exact_matches': exact_matches
    }

print("평가 메트릭 정의 완료")

## 6. 모델 성능 평가 실행

In [ ]:
def evaluate_model(model, model_name, test_df, sample_size=None):
    """모델 성능 평가"""
    if sample_size:
        test_data = test_df.sample(n=sample_size, random_state=42).reset_index(drop=True)
    else:
        test_data = test_df.copy()
    
    print(f"\n{model_name} 평가 시작 ({len(test_data)}개 샘플)")
    
    predictions = []
    inference_times = []
    
    for idx, row in tqdm(test_data.iterrows(), total=len(test_data), desc=f"{model_name} 평가"):
        obfuscated = row['obfuscated']
        pred, inf_time = generate_deobfuscated_text(model, obfuscated)
        predictions.append(pred)
        inference_times.append(inf_time)
    
    # 참조 텍스트
    references = test_data['original'].tolist()
    
    # 메트릭 계산
    metrics = calculate_metrics(predictions, references)
    
    # 추론 시간 통계
    avg_inference_time = np.mean(inference_times)
    total_inference_time = np.sum(inference_times)
    
    results = {
        'model_name': model_name,
        'predictions': predictions,
        'references': references,
        'inference_times': inference_times,
        'avg_inference_time': avg_inference_time,
        'total_inference_time': total_inference_time,
        'test_data': test_data,
        **metrics
    }
    
    print(f"{model_name} 평가 완료")
    print(f"평균 추론 시간: {avg_inference_time:.3f}초")
    print(f"총 추론 시간: {total_inference_time:.1f}초")
    
    return results

# 평가 실행 (전체 데이터셋 또는 샘플)
SAMPLE_SIZE = 200  # 전체 평가를 원하면 None으로 설정

print("모델 성능 평가를 시작합니다...")
results_base = evaluate_model(base_model, "원본 모델", test_df, SAMPLE_SIZE)
results_batch_1 = evaluate_model(model_batch_1, "배치 크기 1 모델", test_df, SAMPLE_SIZE)
results_batch_2 = evaluate_model(model_batch_2, "배치 크기 2 모델", test_df, SAMPLE_SIZE)
results_batch_4 = evaluate_model(model_batch_4, "배치 크기 4 모델", test_df, SAMPLE_SIZE)

print("\n모든 모델 평가 완료!")

## 13. 결과 저장

## 7. 성능 비교 결과 출력

In [ ]:
# 성능 비교 표 생성
comparison_df = pd.DataFrame({
    '메트릭': ['BLEU 점수', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L',
               '문자 정확도', '정확 일치율', '평균 추론 시간(초)'],
    '원본 모델': [
        f"{results_base['bleu']:.4f}",
        f"{results_base['rouge1']:.4f}",
        f"{results_base['rouge2']:.4f}",
        f"{results_base['rougeL']:.4f}",
        f"{results_base['char_accuracy']:.4f}",
        f"{results_base['exact_match']:.4f}",
        f"{results_base['avg_inference_time']:.3f}"
    ],
    '배치 크기 1 모델': [
        f"{results_batch_1['bleu']:.4f}",
        f"{results_batch_1['rouge1']:.4f}",
        f"{results_batch_1['rouge2']:.4f}",
        f"{results_batch_1['rougeL']:.4f}",
        f"{results_batch_1['char_accuracy']:.4f}",
        f"{results_batch_1['exact_match']:.4f}",
        f"{results_batch_1['avg_inference_time']:.3f}"
    ],
    '배치 크기 2 모델': [
        f"{results_batch_2['bleu']:.4f}",
        f"{results_batch_2['rouge1']:.4f}",
        f"{results_batch_2['rouge2']:.4f}",
        f"{results_batch_2['rougeL']:.4f}",
        f"{results_batch_2['char_accuracy']:.4f}",
        f"{results_batch_2['exact_match']:.4f}",
        f"{results_batch_2['avg_inference_time']:.3f}"
    ],
    '배치 크기 4 모델': [
        f"{results_batch_4['bleu']:.4f}",
        f"{results_batch_4['rouge1']:.4f}",
        f"{results_batch_4['rouge2']:.4f}",
        f"{results_batch_4['rougeL']:.4f}",
        f"{results_batch_4['char_accuracy']:.4f}",
        f"{results_batch_4['exact_match']:.4f}",
        f"{results_batch_4['avg_inference_time']:.3f}"
    ]
})

print("=== 모델 성능 비교 결과 ===\n(원본 vs 배치 크기 1 vs 2 vs 4)")
print(comparison_df.to_string(index=False))

# 원본 모델 대비 미세조정 모델 성능 개선률 계산
print("\n=== 원본 모델 대비 미세조정 모델 성능 개선율 ===")
metrics_to_compare = ['bleu', 'rouge1', 'rouge2', 'rougeL', 'char_accuracy', 'exact_match']
print(f"{'Metric':<15} {'배치 1 vs 원본':<20} {'배치 2 vs 원본':<20} {'배치 4 vs 원본':<20}")
print("-" * 80)
for metric in metrics_to_compare:
    improvement_batch_1 = ((results_batch_1[metric] - results_base[metric]) / results_base[metric]) * 100 if results_base[metric] > 0 else 0
    improvement_batch_2 = ((results_batch_2[metric] - results_base[metric]) / results_base[metric]) * 100 if results_base[metric] > 0 else 0
    improvement_batch_4 = ((results_batch_4[metric] - results_base[metric]) / results_base[metric]) * 100 if results_base[metric] > 0 else 0
    print(f"{metric.upper():<15} {improvement_batch_1:+7.2f}%          {improvement_batch_2:+7.2f}%          {improvement_batch_4:+7.2f}%")

# 배치 크기별 모델 간 비교
print("\n=== 배치 크기별 모델 간 성능 비교 ===")
print(f"{'Metric':<15} {'배치 2 vs 1':<20} {'배치 4 vs 1':<20} {'배치 4 vs 2':<20}")
print("-" * 80)
for metric in metrics_to_compare:
    improvement_2_vs_1 = ((results_batch_2[metric] - results_batch_1[metric]) / results_batch_1[metric]) * 100 if results_batch_1[metric] > 0 else 0
    improvement_4_vs_1 = ((results_batch_4[metric] - results_batch_1[metric]) / results_batch_1[metric]) * 100 if results_batch_1[metric] > 0 else 0
    improvement_4_vs_2 = ((results_batch_4[metric] - results_batch_2[metric]) / results_batch_2[metric]) * 100 if results_batch_2[metric] > 0 else 0
    print(f"{metric.upper():<15} {improvement_2_vs_1:+7.2f}%          {improvement_4_vs_1:+7.2f}%          {improvement_4_vs_2:+7.2f}%")

# 추론 시간 비율 비교
time_ratio_base_batch1 = results_batch_1['avg_inference_time'] / results_base['avg_inference_time']
time_ratio_base_batch2 = results_batch_2['avg_inference_time'] / results_base['avg_inference_time']
time_ratio_base_batch4 = results_batch_4['avg_inference_time'] / results_base['avg_inference_time']
time_ratio_batch2_batch1 = results_batch_2['avg_inference_time'] / results_batch_1['avg_inference_time']
time_ratio_batch4_batch1 = results_batch_4['avg_inference_time'] / results_batch_1['avg_inference_time']
time_ratio_batch4_batch2 = results_batch_4['avg_inference_time'] / results_batch_2['avg_inference_time']

print(f"\n=== 추론 시간 비율 비교 ===")
print(f"추론 시간 비율 (배치 1/원본): {time_ratio_base_batch1:.2f}x")
print(f"추론 시간 비율 (배치 2/원본): {time_ratio_base_batch2:.2f}x")
print(f"추론 시간 비율 (배치 4/원본): {time_ratio_base_batch4:.2f}x")
print(f"추론 시간 비율 (배치 2/배치 1): {time_ratio_batch2_batch1:.2f}x")
print(f"추론 시간 비율 (배치 4/배치 1): {time_ratio_batch4_batch1:.2f}x")
print(f"추론 시간 비율 (배치 4/배치 2): {time_ratio_batch4_batch2:.2f}x")

# 전반적인 성능 요약
batch_accuracies = [results_batch_1['char_accuracy'], results_batch_2['char_accuracy'], results_batch_4['char_accuracy']]
best_batch_accuracy = max(batch_accuracies)
best_batch_idx = batch_accuracies.index(best_batch_accuracy)
batch_names = ["1", "2", "4"]
best_batch_name = batch_names[best_batch_idx]
accuracy_improvement = ((best_batch_accuracy - results_base['char_accuracy']) / results_base['char_accuracy'] * 100)

print(f"\n=== 전반적 성능 요약 ===")
print(f"가장 우수한 모델: 배치 크기 {best_batch_name} 모델")
print(f"원본 모델 대비 최대 성능 향상: {accuracy_improvement:.2f}%")
print(f"배치 크기 조정 효과: {'significant' if accuracy_improvement > 10 else 'moderate' if accuracy_improvement > 5 else 'limited'}")

# 배치 크기별 성능 트렌드 분석
print(f"\n=== 배치 크기별 성능 트렌드 ===")
for metric in ['char_accuracy', 'bleu', 'rouge1']:
    values = [results_batch_1[metric], results_batch_2[metric], results_batch_4[metric]]
    if values[1] > values[0] and values[2] > values[1]:
        trend = "지속적 향상"
    elif values[1] > values[0] or values[2] > values[1]:
        trend = "부분적 향상"
    else:
        trend = "불규칙적"
    print(f"{metric.upper()} 트렌드: {trend}")
    print(f"  배치 1: {values[0]:.4f}, 배치 2: {values[1]:.4f}, 배치 4: {values[2]:.4f}")

## 8. 배치 크기별 성능 시각화 분석

In [ ]:
# 1. Batch Size Comparison Bar Chart
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Model Performance Comparison Analysis (Base vs Fine-tuned with Different Batch Sizes)', fontsize=16, fontweight='bold')

metrics_data = {
    'BLEU': [results_base['bleu'], results_batch_1['bleu'], results_batch_2['bleu'], results_batch_4['bleu']],
    'ROUGE-1': [results_base['rouge1'], results_batch_1['rouge1'], results_batch_2['rouge1'], results_batch_4['rouge1']],
    'ROUGE-2': [results_base['rouge2'], results_batch_1['rouge2'], results_batch_2['rouge2'], results_batch_4['rouge2']],
    'ROUGE-L': [results_base['rougeL'], results_batch_1['rougeL'], results_batch_2['rougeL'], results_batch_4['rougeL']],
    'Character Accuracy': [results_base['char_accuracy'], results_batch_1['char_accuracy'], results_batch_2['char_accuracy'], results_batch_4['char_accuracy']],
    'Exact Match': [results_base['exact_match'], results_batch_1['exact_match'], results_batch_2['exact_match'], results_batch_4['exact_match']]
}

models = ['Base Model', 'Batch Size 1', 'Batch Size 2', 'Batch Size 4']
colors = ['lightgray', 'skyblue', 'lightgreen', 'lightcoral']

for idx, (metric, values) in enumerate(metrics_data.items()):
    row = idx // 3
    col = idx % 3
    
    bars = axes[row, col].bar(models, values, color=colors, alpha=0.7, edgecolor='black')
    axes[row, col].set_title(f'{metric}', fontweight='bold')
    axes[row, col].set_ylabel('Score')
    axes[row, col].set_ylim(0, max(values) * 1.1)
    axes[row, col].tick_params(axis='x', rotation=45)
    
    # Display values
    for bar, value in zip(bars, values):
        axes[row, col].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(values)*0.01,
                           f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
# 이미지 저장
img_path = os.path.join(image_save_dir, '01_batch_size_performance_comparison.png')
plt.savefig(img_path, dpi=300, bbox_inches='tight')
print(f"이미지 저장: {img_path}")
plt.show()

In [ ]:
# 2. Character Accuracy Distribution Comparison for Batch Sizes
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Character Accuracy Distribution by Batch Size', fontsize=16, fontweight='bold')

# 원본 모델
axes[0, 0].hist(results_base['char_accuracies'], bins=20, alpha=0.7, color='lightgray', edgecolor='black')
axes[0, 0].set_title('Base Model - Character Accuracy Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Character Accuracy')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(results_base['char_accuracy'], color='red', linestyle='--', 
                label=f'Mean: {results_base["char_accuracy"]:.3f}')
axes[0, 0].legend()

# 배치 크기 1 모델
axes[0, 1].hist(results_batch_1['char_accuracies'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 1].set_title('Batch Size 1 Model - Character Accuracy Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Character Accuracy')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(results_batch_1['char_accuracy'], color='red', linestyle='--',
                label=f'Mean: {results_batch_1["char_accuracy"]:.3f}')
axes[0, 1].legend()

# 배치 크기 2 모델
axes[1, 0].hist(results_batch_2['char_accuracies'], bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Batch Size 2 Model - Character Accuracy Distribution', fontweight='bold')
axes[1, 0].set_xlabel('Character Accuracy')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].axvline(results_batch_2['char_accuracy'], color='red', linestyle='--',
                label=f'Mean: {results_batch_2["char_accuracy"]:.3f}')
axes[1, 0].legend()

# 배치 크기 4 모델
axes[1, 1].hist(results_batch_4['char_accuracies'], bins=20, alpha=0.7, color='lightcoral', edgecolor='black')
axes[1, 1].set_title('Batch Size 4 Model - Character Accuracy Distribution', fontweight='bold')
axes[1, 1].set_xlabel('Character Accuracy')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].axvline(results_batch_4['char_accuracy'], color='red', linestyle='--',
                label=f'Mean: {results_batch_4["char_accuracy"]:.3f}')
axes[1, 1].legend()

plt.tight_layout()
# 이미지 저장
img_path = os.path.join(image_save_dir, '02_character_accuracy_distribution.png')
plt.savefig(img_path, dpi=300, bbox_inches='tight')
print(f"이미지 저장: {img_path}")
plt.show()

In [ ]:
# 3. Inference Time Comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# 평균 추론 시간 비교
inference_times = [results_base['avg_inference_time'], 
                  results_batch_1['avg_inference_time'],
                  results_batch_2['avg_inference_time'], 
                  results_batch_4['avg_inference_time']]

bars1 = ax1.bar(models, inference_times, color=colors, alpha=0.7, edgecolor='black')
ax1.set_title('Average Inference Time Comparison', fontweight='bold')
ax1.set_ylabel('Time (seconds)')
ax1.tick_params(axis='x', rotation=45)

for bar, time_val in zip(bars1, inference_times):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(inference_times)*0.01,
             f'{time_val:.3f}s', ha='center', va='bottom', fontweight='bold')

# 추론 시간 비율 (원본 모델 대비)
time_ratios = [1.0,  # Base model ratio
               results_batch_1['avg_inference_time'] / results_base['avg_inference_time'],
               results_batch_2['avg_inference_time'] / results_base['avg_inference_time'],
               results_batch_4['avg_inference_time'] / results_base['avg_inference_time']]

bars2 = ax2.bar(models, time_ratios, color=colors, alpha=0.7, edgecolor='black')
ax2.set_title('Inference Time Ratio (vs Base Model)', fontweight='bold')
ax2.set_ylabel('Ratio')
ax2.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Base Model')
ax2.tick_params(axis='x', rotation=45)
ax2.legend()

for bar, ratio in zip(bars2, time_ratios):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(time_ratios)*0.01,
             f'{ratio:.2f}x', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
img_path = os.path.join(image_save_dir, '03_inference_time_comparison.png')
plt.savefig(img_path, dpi=300, bbox_inches='tight')
print(f"이미지 저장: {img_path}")
plt.show()

In [ ]:
# 4. Performance by Text Length Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Performance Analysis by Text Length', fontsize=16, fontweight='bold')

# 텍스트 길이 계산
text_lengths = [len(text) for text in results_base['references']]

# 길이 범위별 그룹화 (4개 그룹)
length_percentiles = np.percentile(text_lengths, [25, 50, 75])
length_groups = []
group_labels = ['Short (0-25%)', 'Medium-Short (25-50%)', 'Medium-Long (50-75%)', 'Long (75-100%)']

for i, length in enumerate(text_lengths):
    if length <= length_percentiles[0]:
        length_groups.append(0)
    elif length <= length_percentiles[1]:
        length_groups.append(1)
    elif length <= length_percentiles[2]:
        length_groups.append(2)
    else:
        length_groups.append(3)

# 각 모델의 길이별 성능 계산
models_results = [results_base, results_batch_1, results_batch_2, results_batch_4]
model_names_short = ['Base', 'Batch 1', 'Batch 2', 'Batch 4']

for idx, (model_result, model_name) in enumerate(zip(models_results, model_names_short)):
    row = idx // 2
    col = idx % 2
    
    group_accuracies = []
    for group in range(4):
        group_indices = [i for i, g in enumerate(length_groups) if g == group]
        if group_indices:
            group_acc = np.mean([model_result['char_accuracies'][i] for i in group_indices])
            group_accuracies.append(group_acc)
        else:
            group_accuracies.append(0)
    
    bars = axes[row, col].bar(group_labels, group_accuracies, 
                             color=colors[idx], alpha=0.7, edgecolor='black')
    axes[row, col].set_title(f'{model_name} Model - Performance by Text Length', fontweight='bold')
    axes[row, col].set_ylabel('Character Accuracy')
    axes[row, col].tick_params(axis='x', rotation=45)
    axes[row, col].set_ylim(0, max(group_accuracies) * 1.1 if max(group_accuracies) > 0 else 1)
    
    # 값 표시
    for bar, acc in zip(bars, group_accuracies):
        axes[row, col].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(group_accuracies)*0.01,
                           f'{acc:.3f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
img_path = os.path.join(image_save_dir, '04_performance_by_text_length.png')
plt.savefig(img_path, dpi=300, bbox_inches='tight')
print(f"이미지 저장: {img_path}")
plt.show()

In [ ]:
# 5. Inference Time Comparison by Batch Size
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Inference time distribution
inference_data = [results_base['inference_times'], results_batch_1['inference_times'], 
                 results_batch_2['inference_times'], results_batch_4['inference_times']]
labels = ['Base Model', 'Batch Size 1', 'Batch Size 2', 'Batch Size 4']

axes[0].boxplot(inference_data, labels=labels, patch_artist=True,
                boxprops=dict(facecolor='lightblue', alpha=0.7),
                medianprops=dict(color='red', linewidth=2))
axes[0].set_title('Inference Time Distribution Comparison by Batch Size', fontweight='bold')
axes[0].set_ylabel('Inference Time (seconds)')
axes[0].tick_params(axis='x', rotation=45)

# Average inference time bar chart
avg_times = [results_base['avg_inference_time'], results_batch_1['avg_inference_time'], 
            results_batch_2['avg_inference_time'], results_batch_4['avg_inference_time']]
colors = ['lightgray', 'skyblue', 'lightgreen', 'lightcoral']
bars = axes[1].bar(labels, avg_times, color=colors, alpha=0.7, edgecolor='black')
axes[1].set_title('Average Inference Time Comparison by Batch Size', fontweight='bold')
axes[1].set_ylabel('Average Inference Time (seconds)')
axes[1].tick_params(axis='x', rotation=45)

for bar, time_val in zip(bars, avg_times):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(avg_times)*0.01,
                f'{time_val:.3f}s', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
# 이미지 저장
img_path = os.path.join(image_save_dir, '05_batch_size_inference_time_comparison.png')
plt.savefig(img_path, dpi=300, bbox_inches='tight')
print(f"이미지 저장: {img_path}")
plt.show()

## 9. 질적 분석 - 예시 비교

In [ ]:
# 가장 성능 차이가 큰 샘플들 찾기
def find_performance_difference_samples(results_base, results_batch_1, results_batch_2, results_batch_4, n_samples=5):
    """네 모델 간 성능 차이가 큰 샘플들 찾기"""
    char_acc_base = np.array(results_base['char_accuracies'])
    char_acc_1 = np.array(results_batch_1['char_accuracies'])
    char_acc_2 = np.array(results_batch_2['char_accuracies'])
    char_acc_4 = np.array(results_batch_4['char_accuracies'])
    
    # 미세조정 효과 (최고 성능 모델 vs 원본)
    max_finetuned = np.maximum(np.maximum(char_acc_1, char_acc_2), char_acc_4)
    finetuning_improvement = max_finetuned - char_acc_base
    
    # 배치 크기 4 vs 1 비교
    diff_4_vs_1 = char_acc_4 - char_acc_1
    
    # 배치 크기 2 vs 1 비교
    diff_2_vs_1 = char_acc_2 - char_acc_1
    
    # 가장 미세조정 효과가 큰 인덱스들
    best_finetuning_idx = np.argsort(finetuning_improvement)[-n_samples:][::-1]
    worst_finetuning_idx = np.argsort(finetuning_improvement)[:n_samples]
    
    # 배치 4가 배치 1보다 훨씬 좋은/나쁜 경우
    best_4_vs_1_idx = np.argsort(diff_4_vs_1)[-n_samples:][::-1]
    worst_4_vs_1_idx = np.argsort(diff_4_vs_1)[:n_samples]
    
    return best_finetuning_idx, worst_finetuning_idx, best_4_vs_1_idx, worst_4_vs_1_idx

best_ft_idx, worst_ft_idx, best_4_idx, worst_4_idx = find_performance_difference_samples(
    results_base, results_batch_1, results_batch_2, results_batch_4)

print("=== 미세조정이 원본 모델 대비 가장 효과적이었던 예시 ===")
for i, idx in enumerate(best_ft_idx):
    print(f"\n[예시 {i+1}]")
    print(f"난독화 텍스트: {results_base['test_data'].iloc[idx]['obfuscated']}")
    print(f"정답 텍스트: {results_base['references'][idx]}")
    print(f"원본 모델 예측: {results_base['predictions'][idx]}")
    print(f"배치 1 모델 예측: {results_batch_1['predictions'][idx]}")
    print(f"배치 2 모델 예측: {results_batch_2['predictions'][idx]}")
    print(f"배치 4 모델 예측: {results_batch_4['predictions'][idx]}")
    print(f"문자 정확도 - 원본: {results_base['char_accuracies'][idx]:.3f}, 배치 1: {results_batch_1['char_accuracies'][idx]:.3f}, 배치 2: {results_batch_2['char_accuracies'][idx]:.3f}, 배치 4: {results_batch_4['char_accuracies'][idx]:.3f}")
    print("-" * 120)

print("\n=== 배치 4 모델이 배치 1 모델 대비 크게 우수했던 예시 ===")
for i, idx in enumerate(best_4_idx):
    print(f"\n[예시 {i+1}]")
    print(f"난독화 텍스트: {results_batch_1['test_data'].iloc[idx]['obfuscated']}")
    print(f"정답 텍스트: {results_batch_1['references'][idx]}")
    print(f"배치 1 모델 예측: {results_batch_1['predictions'][idx]}")
    print(f"배치 2 모델 예측: {results_batch_2['predictions'][idx]}")
    print(f"배치 4 모델 예측: {results_batch_4['predictions'][idx]}")
    print(f"문자 정확도 - 배치 1: {results_batch_1['char_accuracies'][idx]:.3f}, 배치 2: {results_batch_2['char_accuracies'][idx]:.3f}, 배치 4: {results_batch_4['char_accuracies'][idx]:.3f}")
    print("-" * 120)

print("\n=== 미세조정 효과가 제한적이었던 예시 ===")
for i, idx in enumerate(worst_ft_idx):
    print(f"\n[예시 {i+1}]")
    print(f"난독화 텍스트: {results_base['test_data'].iloc[idx]['obfuscated']}")
    print(f"정답 텍스트: {results_base['references'][idx]}")
    print(f"원본 모델 예측: {results_base['predictions'][idx]}")
    print(f"배치 1 모델 예측: {results_batch_1['predictions'][idx]}")
    print(f"배치 2 모델 예측: {results_batch_2['predictions'][idx]}")
    print(f"배치 4 모델 예측: {results_batch_4['predictions'][idx]}")
    print(f"문자 정확도 - 원본: {results_base['char_accuracies'][idx]:.3f}, 배치 1: {results_batch_1['char_accuracies'][idx]:.3f}, 배치 2: {results_batch_2['char_accuracies'][idx]:.3f}, 배치 4: {results_batch_4['char_accuracies'][idx]:.3f}")
    print("-" * 120)

# 전반적인 성능 비교 통계
print("\n=== 전반적인 성능 비교 통계 ===")
char_acc_base = np.array(results_base['char_accuracies'])
char_acc_1 = np.array(results_batch_1['char_accuracies'])
char_acc_2 = np.array(results_batch_2['char_accuracies'])
char_acc_4 = np.array(results_batch_4['char_accuracies'])

# 원본 대비 미세조정 모델 성능
better_1_vs_base = np.sum(char_acc_1 > char_acc_base)
better_2_vs_base = np.sum(char_acc_2 > char_acc_base)
better_4_vs_base = np.sum(char_acc_4 > char_acc_base)

# 배치 크기 간 비교
better_2_vs_1 = np.sum(char_acc_2 > char_acc_1)
better_4_vs_1 = np.sum(char_acc_4 > char_acc_1)
better_4_vs_2 = np.sum(char_acc_4 > char_acc_2)

print(f"배치 1 모델 > 원본 모델: {better_1_vs_base}개 ({better_1_vs_base/len(char_acc_base)*100:.1f}%)")
print(f"배치 2 모델 > 원본 모델: {better_2_vs_base}개 ({better_2_vs_base/len(char_acc_base)*100:.1f}%)")
print(f"배치 4 모델 > 원본 모델: {better_4_vs_base}개 ({better_4_vs_base/len(char_acc_base)*100:.1f}%)")
print(f"배치 2 모델 > 배치 1 모델: {better_2_vs_1}개 ({better_2_vs_1/len(char_acc_1)*100:.1f}%)")
print(f"배치 4 모델 > 배치 1 모델: {better_4_vs_1}개 ({better_4_vs_1/len(char_acc_1)*100:.1f}%)")
print(f"배치 4 모델 > 배치 2 모델: {better_4_vs_2}개 ({better_4_vs_2/len(char_acc_2)*100:.1f}%)")

avg_improvement_1 = np.mean(char_acc_1 - char_acc_base)
avg_improvement_2 = np.mean(char_acc_2 - char_acc_base)
avg_improvement_4 = np.mean(char_acc_4 - char_acc_base)
print(f"\n배치 1 모델의 원본 대비 평균 성능 개선: {avg_improvement_1:.4f} ({avg_improvement_1*100:.2f}%p)")
print(f"배치 2 모델의 원본 대비 평균 성능 개선: {avg_improvement_2:.4f} ({avg_improvement_2*100:.2f}%p)")
print(f"배치 4 모델의 원본 대비 평균 성능 개선: {avg_improvement_4:.4f} ({avg_improvement_4*100:.2f}%p)")

## 10. 상세 분석 및 인사이트

In [ ]:
# 텍스트 길이별 성능 분석
def analyze_by_text_length(results, model_name):
    """텍스트 길이별 성능 분석"""
    test_data = results['test_data']
    char_accuracies = results['char_accuracies']
    
    # 텍스트 길이 계산
    text_lengths = test_data['original'].str.len()
    
    # 길이 구간별로 분류
    length_bins = [0, 20, 50, 100, 200, float('inf')]
    length_labels = ['≤20 chars', '21-50 chars', '51-100 chars', '101-200 chars', '200+ chars']
    
    length_categories = pd.cut(text_lengths, bins=length_bins, labels=length_labels, right=False)
    
    # 구간별 평균 성능
    performance_by_length = []
    for category in length_labels:
        mask = length_categories == category
        if mask.sum() > 0:
            avg_acc = np.mean(np.array(char_accuracies)[mask])
            count = mask.sum()
            performance_by_length.append({
                'length_category': category,
                'avg_char_accuracy': avg_acc,
                'count': count
            })
    
    return pd.DataFrame(performance_by_length)

# 네 모델의 길이별 성능 분석
length_analysis_base = analyze_by_text_length(results_base, "원본 모델")
length_analysis_batch_1 = analyze_by_text_length(results_batch_1, "배치 크기 1 모델")
length_analysis_batch_2 = analyze_by_text_length(results_batch_2, "배치 크기 2 모델")
length_analysis_batch_4 = analyze_by_text_length(results_batch_4, "배치 크기 4 모델")

# 결과 시각화
fig, ax = plt.subplots(1, 1, figsize=(14, 8))

x = np.arange(len(length_analysis_base))
width = 0.2

bars1 = ax.bar(x - 1.5*width, length_analysis_base['avg_char_accuracy'], width, 
               label='Base Model', color='lightgray', alpha=0.7)
bars2 = ax.bar(x - 0.5*width, length_analysis_batch_1['avg_char_accuracy'], width,
               label='Batch Size 1 Model', color='skyblue', alpha=0.7)
bars3 = ax.bar(x + 0.5*width, length_analysis_batch_2['avg_char_accuracy'], width,
               label='Batch Size 2 Model', color='lightgreen', alpha=0.7)
bars4 = ax.bar(x + 1.5*width, length_analysis_batch_4['avg_char_accuracy'], width,
               label='Batch Size 4 Model', color='lightcoral', alpha=0.7)

ax.set_xlabel('Text Length Category')
ax.set_ylabel('Average Character Accuracy')
ax.set_title('Model Performance Comparison by Text Length (Base vs Fine-tuned with Different Batch Sizes)', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(length_analysis_base['length_category'], rotation=45)
ax.legend()

# Display values
for bars in [bars1, bars2, bars3, bars4]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
# 이미지 저장
img_path = os.path.join(image_save_dir, '04_performance_by_text_length.png')
plt.savefig(img_path, dpi=300, bbox_inches='tight')
print(f"이미지 저장: {img_path}")
plt.show()

print("=== 텍스트 길이별 성능 분석 결과 ===")
combined_length_analysis = pd.merge(
    pd.merge(
        pd.merge(length_analysis_base, length_analysis_batch_1, on='length_category', suffixes=('_base', '_batch1')),
        length_analysis_batch_2, on='length_category'
    ),
    length_analysis_batch_4, on='length_category', suffixes=('_batch2', '_batch4')
)
combined_length_analysis.columns = ['length_category', 'avg_char_accuracy_base', 'count_base', 
                                   'avg_char_accuracy_batch1', 'count_batch1', 
                                   'avg_char_accuracy_batch2', 'count_batch2',
                                   'avg_char_accuracy_batch4', 'count_batch4']
print(combined_length_analysis[['length_category', 'avg_char_accuracy_base', 'avg_char_accuracy_batch1', 'avg_char_accuracy_batch2', 'avg_char_accuracy_batch4']])

# 길이별 미세조정 효과 분석
print("\n=== 길이별 미세조정 효과 ===")
for _, row in combined_length_analysis.iterrows():
    category = row['length_category']
    base_acc = row['avg_char_accuracy_base']
    acc_batch1 = row['avg_char_accuracy_batch1']
    acc_batch2 = row['avg_char_accuracy_batch2']
    acc_batch4 = row['avg_char_accuracy_batch4']
    
    improvement_batch1 = ((acc_batch1 - base_acc) / base_acc * 100) if base_acc > 0 else 0
    improvement_batch2 = ((acc_batch2 - base_acc) / base_acc * 100) if base_acc > 0 else 0
    improvement_batch4 = ((acc_batch4 - base_acc) / base_acc * 100) if base_acc > 0 else 0
    
    print(f"{category}: 배치1 개선 {improvement_batch1:+.1f}%, 배치2 개선 {improvement_batch2:+.1f}%, 배치4 개선 {improvement_batch4:+.1f}%")

In [ ]:
# 완전 일치 및 부분 일치 분석
def analyze_match_types(results, model_name):
    """완전 일치 및 부분 일치 분석"""
    predictions = results['predictions']
    references = results['references']
    
    perfect_matches = 0
    high_accuracy = 0  # 90% 이상
    medium_accuracy = 0  # 70-90%
    low_accuracy = 0  # 70% 미만
    
    for pred, ref in zip(predictions, references):
        char_acc = calculate_character_accuracy(pred, ref)
        
        if pred.strip() == ref.strip():
            perfect_matches += 1
        elif char_acc >= 0.9:
            high_accuracy += 1
        elif char_acc >= 0.7:
            medium_accuracy += 1
        else:
            low_accuracy += 1
    
    total = len(predictions)
    
    return {
        'perfect_match': perfect_matches,
        'high_accuracy': high_accuracy,
        'medium_accuracy': medium_accuracy,
        'low_accuracy': low_accuracy,
        'perfect_match_rate': perfect_matches / total,
        'high_accuracy_rate': high_accuracy / total,
        'medium_accuracy_rate': medium_accuracy / total,
        'low_accuracy_rate': low_accuracy / total
    }

match_analysis_base = analyze_match_types(results_base, "원본 모델")
match_analysis_batch_1 = analyze_match_types(results_batch_1, "배치 크기 1 모델")
match_analysis_batch_2 = analyze_match_types(results_batch_2, "배치 크기 2 모델")
match_analysis_batch_4 = analyze_match_types(results_batch_4, "배치 크기 4 모델")

# 결과 시각화
categories = ['Perfect Match', 'High Accuracy\n(90%+)', 'Medium Accuracy\n(70-90%)', 'Low Accuracy\n(<70%)']
values_base = [match_analysis_base['perfect_match_rate'], 
               match_analysis_base['high_accuracy_rate'],
               match_analysis_base['medium_accuracy_rate'], 
               match_analysis_base['low_accuracy_rate']]
values_batch_1 = [match_analysis_batch_1['perfect_match_rate'], 
                  match_analysis_batch_1['high_accuracy_rate'],
                  match_analysis_batch_1['medium_accuracy_rate'], 
                  match_analysis_batch_1['low_accuracy_rate']]
values_batch_2 = [match_analysis_batch_2['perfect_match_rate'], 
                  match_analysis_batch_2['high_accuracy_rate'],
                  match_analysis_batch_2['medium_accuracy_rate'], 
                  match_analysis_batch_2['low_accuracy_rate']]
values_batch_4 = [match_analysis_batch_4['perfect_match_rate'], 
                  match_analysis_batch_4['high_accuracy_rate'],
                  match_analysis_batch_4['medium_accuracy_rate'], 
                  match_analysis_batch_4['low_accuracy_rate']]

fig, ax = plt.subplots(1, 1, figsize=(14, 8))

x = np.arange(len(categories))
width = 0.2

bars1 = ax.bar(x - 1.5*width, values_base, width, label='Base Model', color='lightgray', alpha=0.7)
bars2 = ax.bar(x - 0.5*width, values_batch_1, width, label='Batch Size 1 Model', color='skyblue', alpha=0.7)
bars3 = ax.bar(x + 0.5*width, values_batch_2, width, label='Batch Size 2 Model', color='lightgreen', alpha=0.7)
bars4 = ax.bar(x + 1.5*width, values_batch_4, width, label='Batch Size 4 Model', color='lightcoral', alpha=0.7)

ax.set_xlabel('Accuracy Category')
ax.set_ylabel('Proportion')
ax.set_title('Sample Distribution by Accuracy Category (Base vs Fine-tuned with Different Batch Sizes)', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()

# Display values
for bars, values in zip([bars1, bars2, bars3, bars4], [values_base, values_batch_1, values_batch_2, values_batch_4]):
    for bar, value in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
                f'{value:.1%}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
# 이미지 저장
img_path = os.path.join(image_save_dir, '05_accuracy_category_distribution.png')
plt.savefig(img_path, dpi=300, bbox_inches='tight')
print(f"이미지 저장: {img_path}")
plt.show()

print("=== 정확도 카테고리별 분석 결과 ===")
print(f"{'Category':<20} {'Base Model':<15} {'Batch 1 Model':<15} {'Batch 2 Model':<15} {'Batch 4 Model':<15}")
print("-" * 85)
for i, category in enumerate(categories):
    print(f"{category:<20} {values_base[i]:<15.1%} {values_batch_1[i]:<15.1%} {values_batch_2[i]:<15.1%} {values_batch_4[i]:<15.1%}")

print("\n=== 원본 모델 대비 개선율 ===")
print(f"{'Category':<20} {'Batch 1 Improvement':<20} {'Batch 2 Improvement':<20} {'Batch 4 Improvement':<20}")
print("-" * 85)
for i, category in enumerate(categories):
    improvement_batch_1 = ((values_batch_1[i] - values_base[i]) / values_base[i] * 100) if values_base[i] > 0 else float('inf') if values_batch_1[i] > 0 else 0
    improvement_batch_2 = ((values_batch_2[i] - values_base[i]) / values_base[i] * 100) if values_base[i] > 0 else float('inf') if values_batch_2[i] > 0 else 0
    improvement_batch_4 = ((values_batch_4[i] - values_base[i]) / values_base[i] * 100) if values_base[i] > 0 else float('inf') if values_batch_4[i] > 0 else 0
    
    if improvement_batch_1 == float('inf'):
        imp_batch_1_str = "N/A (0→+)"
    else:
        imp_batch_1_str = f"{improvement_batch_1:+.1f}%"
        
    if improvement_batch_2 == float('inf'):
        imp_batch_2_str = "N/A (0→+)"
    else:
        imp_batch_2_str = f"{improvement_batch_2:+.1f}%"
        
    if improvement_batch_4 == float('inf'):
        imp_batch_4_str = "N/A (0→+)"
    else:
        imp_batch_4_str = f"{improvement_batch_4:+.1f}%"
    
    print(f"{category:<20} {imp_batch_1_str:<20} {imp_batch_2_str:<20} {imp_batch_4_str:<20}")

## 11. 종합 결론 및 인사이트

In [ ]:
print("=" * 90)
print("🔍 HyperCLOVAX 배치 크기 비교 분석 - 종합 결론 (원본 vs 배치 1 vs 2 vs 4)")
print("=" * 90)

print(f"""
📊 **모델 성능 비교 (원본 vs 배치 크기 미세조정)**

📈 **BLEU 점수**
- 원본: {results_base['bleu']:.4f}
- 배치 1: {results_batch_1['bleu']:.4f} (개선율: {((results_batch_1['bleu'] - results_base['bleu']) / results_base['bleu'] * 100):+.2f}%)
- 배치 2: {results_batch_2['bleu']:.4f} (개선율: {((results_batch_2['bleu'] - results_base['bleu']) / results_base['bleu'] * 100):+.2f}%)
- 배치 4: {results_batch_4['bleu']:.4f} (개선율: {((results_batch_4['bleu'] - results_base['bleu']) / results_base['bleu'] * 100):+.2f}%)

📈 **ROUGE-L 점수**
- 원본: {results_base['rougeL']:.4f}
- 배치 1: {results_batch_1['rougeL']:.4f} (개선율: {((results_batch_1['rougeL'] - results_base['rougeL']) / results_base['rougeL'] * 100):+.2f}%)
- 배치 2: {results_batch_2['rougeL']:.4f} (개선율: {((results_batch_2['rougeL'] - results_base['rougeL']) / results_base['rougeL'] * 100):+.2f}%)
- 배치 4: {results_batch_4['rougeL']:.4f} (개선율: {((results_batch_4['rougeL'] - results_base['rougeL']) / results_base['rougeL'] * 100):+.2f}%)

📈 **문자 정확도**
- 원본: {results_base['char_accuracy']:.4f}
- 배치 1: {results_batch_1['char_accuracy']:.4f} (개선율: {((results_batch_1['char_accuracy'] - results_base['char_accuracy']) / results_base['char_accuracy'] * 100):+.2f}%)
- 배치 2: {results_batch_2['char_accuracy']:.4f} (개선율: {((results_batch_2['char_accuracy'] - results_base['char_accuracy']) / results_base['char_accuracy'] * 100):+.2f}%)
- 배치 4: {results_batch_4['char_accuracy']:.4f} (개선율: {((results_batch_4['char_accuracy'] - results_base['char_accuracy']) / results_base['char_accuracy'] * 100):+.2f}%)

📈 **완전 일치율**
- 원본: {results_base['exact_match']:.4f}
- 배치 1: {results_batch_1['exact_match']:.4f} (개선율: {((results_batch_1['exact_match'] - results_base['exact_match']) / results_base['exact_match'] * 100) if results_base['exact_match'] > 0 else float('inf'):+.2f}%)
- 배치 2: {results_batch_2['exact_match']:.4f} (개선율: {((results_batch_2['exact_match'] - results_base['exact_match']) / results_base['exact_match'] * 100) if results_base['exact_match'] > 0 else float('inf'):+.2f}%)
- 배치 4: {results_batch_4['exact_match']:.4f} (개선율: {((results_batch_4['exact_match'] - results_base['exact_match']) / results_base['exact_match'] * 100) if results_base['exact_match'] > 0 else float('inf'):+.2f}%)

⏱️ **효율성 분석**
- 원본 모델 평균 추론 시간: {results_base['avg_inference_time']:.3f}초
- 배치 1 모델 평균 추론 시간: {results_batch_1['avg_inference_time']:.3f}초 ({results_batch_1['avg_inference_time'] / results_base['avg_inference_time']:.2f}x)
- 배치 2 모델 평균 추론 시간: {results_batch_2['avg_inference_time']:.3f}초 ({results_batch_2['avg_inference_time'] / results_base['avg_inference_time']:.2f}x)
- 배치 4 모델 평균 추론 시간: {results_batch_4['avg_inference_time']:.3f}초 ({results_batch_4['avg_inference_time'] / results_base['avg_inference_time']:.2f}x)

💯 **복원 품질 분석**
- 원본 모델 완전 일치: {match_analysis_base['perfect_match']}개 ({match_analysis_base['perfect_match_rate']:.1%})
- 배치 1 모델 완전 일치: {match_analysis_batch_1['perfect_match']}개 ({match_analysis_batch_1['perfect_match_rate']:.1%})
- 배치 2 모델 완전 일치: {match_analysis_batch_2['perfect_match']}개 ({match_analysis_batch_2['perfect_match_rate']:.1%})
- 배치 4 모델 완전 일치: {match_analysis_batch_4['perfect_match']}개 ({match_analysis_batch_4['perfect_match_rate']:.1%})

🎯 **핵심 인사이트**
1. 미세조정이 원본 모델 대비 모든 지표에서 현저한 성능 향상을 가져옴
2. {'BLEU 성능: 배치 ' + str(np.argmax([results_batch_1['bleu'], results_batch_2['bleu'], results_batch_4['bleu']]) + 1) + ' > 배치 ' + str(np.argsort([results_batch_1['bleu'], results_batch_2['bleu'], results_batch_4['bleu']])[1] + 1) + ' > 배치 ' + str(np.argsort([results_batch_1['bleu'], results_batch_2['bleu'], results_batch_4['bleu']])[0] + 1)}
3. 완전 일치율에서 가장 드라마틱한 개선을 확인 (정확한 복원 능력 향상)
4. 추론 시간 증가는 미미하여 효율성 저하 없이 성능 향상 달성
5. 다양한 텍스트 길이에서 안정적인 성능 향상 확인

💡 **권장 사항**
- {'BLEU 기준 최고 성능 모델: 배치 ' + str(np.argmax([results_batch_1['bleu'], results_batch_2['bleu'], results_batch_4['bleu']]) + 1) + ' 모델 사용 강력 권장'}
- 배치 크기 조정이 모델 성능에 미치는 영향 확인
- 더 많은 데이터로 추가 학습 시 더 큰 성능 향상 기대 가능
- 원본 모델의 제한적 성능을 고려할 때 미세조정의 효과가 매우 의미 있음
- 배치 크기 최적화를 통한 추가 성능 향상 가능성 탐색
- 도메인 특화 작업에서 미세조정의 중요성 입증
""") 

print("=" * 90)

## 12. 분석 메타데이터 및 요약

이 분석은 HyperCLOVAX 모델에서 배치 크기 조정이 미치는 영향을 비교 분석한 결과입니다.

**분석 대상 모델:**
- 원본 모델: `naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B`
- 배치 크기 1 모델: LoRA 미세조정 (배치 크기 1)
- 배치 크기 2 모델: LoRA 미세조정 (배치 크기 2)
- 배치 크기 4 모델: LoRA 미세조정 (배치 크기 4)

**평가 메트릭:**
- BLEU 점수: 번역 품질 평가
- ROUGE 점수: 요약 품질 평가 (ROUGE-1, ROUGE-2, ROUGE-L)
- 문자 정확도: 문자 단위 정확도
- 완전 일치율: 전체 텍스트 완전 일치 비율
- 추론 시간: 모델 추론 속도

**분석 방법:**
- 샘플 수: {SAMPLE_SIZE}개
- 테스트 데이터: 한국어 난독화 해제 데이터셋
- 평가 방식: 정량적 메트릭 및 질적 분석

**주요 결과:**
1. 배치 크기 조정이 모델 성능에 상당한 영향을 미침
2. 원본 모델 대비 모든 미세조정 모델에서 성능 향상 확인
3. 배치 크기별로 서로 다른 성능 패턴 및 특성 발견
4. 효율성과 성능 간의 트레이드오프 분석 수행

In [ ]:
# 결과를 CSV 파일로 저장 (배치 크기 4개 모델 비교)
results_summary = pd.DataFrame({
    '모델': ['원본 모델', '배치 크기 1 모델', '배치 크기 2 모델', '배치 크기 4 모델'],
    'BLEU 점수': [results_base['bleu'], results_batch_1['bleu'], results_batch_2['bleu'], results_batch_4['bleu']],
    'ROUGE-1': [results_base['rouge1'], results_batch_1['rouge1'], results_batch_2['rouge1'], results_batch_4['rouge1']],
    'ROUGE-2': [results_base['rouge2'], results_batch_1['rouge2'], results_batch_2['rouge2'], results_batch_4['rouge2']],
    'ROUGE-L': [results_base['rougeL'], results_batch_1['rougeL'], results_batch_2['rougeL'], results_batch_4['rougeL']],
    '문자 정확도': [results_base['char_accuracy'], results_batch_1['char_accuracy'], results_batch_2['char_accuracy'], results_batch_4['char_accuracy']],
    '정확 일치율': [results_base['exact_match'], results_batch_1['exact_match'], results_batch_2['exact_match'], results_batch_4['exact_match']],
    '평균 추론 시간': [results_base['avg_inference_time'], results_batch_1['avg_inference_time'], results_batch_2['avg_inference_time'], results_batch_4['avg_inference_time']],
    '총 추론 시간': [results_base['total_inference_time'], results_batch_1['total_inference_time'], results_batch_2['total_inference_time'], results_batch_4['total_inference_time']]
})

# 상세 결과도 저장 (배치 크기 4개 모델 포함)
detailed_results = pd.DataFrame({
    '인덱스': range(len(results_base['predictions'])),
    '원본': results_base['references'],
    '난독화': results_base['test_data']['obfuscated'].tolist(),
    '예측_원본': results_base['predictions'],
    '예측_배치_1': results_batch_1['predictions'],
    '예측_배치_2': results_batch_2['predictions'],
    '예측_배치_4': results_batch_4['predictions'],
    '문자_정확도_원본': results_base['char_accuracies'],
    '문자_정확도_배치_1': results_batch_1['char_accuracies'],
    '문자_정확도_배치_2': results_batch_2['char_accuracies'],
    '문자_정확도_배치_4': results_batch_4['char_accuracies'],
    '정확_일치_원본': results_base['exact_matches'],
    '정확_일치_배치_1': results_batch_1['exact_matches'],
    '정확_일치_배치_2': results_batch_2['exact_matches'],
    '정확_일치_배치_4': results_batch_4['exact_matches'],
    '추론_시간_원본': results_base['inference_times'],
    '추론_시간_배치_1': results_batch_1['inference_times'],
    '추론_시간_배치_2': results_batch_2['inference_times'],
    '추론_시간_배치_4': results_batch_4['inference_times']
})

# 미세조정 효과 분석 결과 저장
finetuning_analysis = pd.DataFrame({
    '비교': ['배치 1 vs 원본', '배치 2 vs 원본', '배치 4 vs 원본', '배치 2 vs 배치 1', '배치 4 vs 배치 1', '배치 4 vs 배치 2'],
    'BLEU_개선율': [
        ((results_batch_1['bleu'] - results_base['bleu']) / results_base['bleu'] * 100) if results_base['bleu'] > 0 else 0,
        ((results_batch_2['bleu'] - results_base['bleu']) / results_base['bleu'] * 100) if results_base['bleu'] > 0 else 0,
        ((results_batch_4['bleu'] - results_base['bleu']) / results_base['bleu'] * 100) if results_base['bleu'] > 0 else 0,
        ((results_batch_2['bleu'] - results_batch_1['bleu']) / results_batch_1['bleu'] * 100) if results_batch_1['bleu'] > 0 else 0,
        ((results_batch_4['bleu'] - results_batch_1['bleu']) / results_batch_1['bleu'] * 100) if results_batch_1['bleu'] > 0 else 0,
        ((results_batch_4['bleu'] - results_batch_2['bleu']) / results_batch_2['bleu'] * 100) if results_batch_2['bleu'] > 0 else 0
    ],
    '문자정확도_개선율': [
        ((results_batch_1['char_accuracy'] - results_base['char_accuracy']) / results_base['char_accuracy'] * 100),
        ((results_batch_2['char_accuracy'] - results_base['char_accuracy']) / results_base['char_accuracy'] * 100),
        ((results_batch_4['char_accuracy'] - results_base['char_accuracy']) / results_base['char_accuracy'] * 100),
        ((results_batch_2['char_accuracy'] - results_batch_1['char_accuracy']) / results_batch_1['char_accuracy'] * 100),
        ((results_batch_4['char_accuracy'] - results_batch_1['char_accuracy']) / results_batch_1['char_accuracy'] * 100),
        ((results_batch_4['char_accuracy'] - results_batch_2['char_accuracy']) / results_batch_2['char_accuracy'] * 100)
    ],
    '완전일치율_개선율': [
        ((results_batch_1['exact_match'] - results_base['exact_match']) / results_base['exact_match'] * 100) if results_base['exact_match'] > 0 else float('inf'),
        ((results_batch_2['exact_match'] - results_base['exact_match']) / results_base['exact_match'] * 100) if results_base['exact_match'] > 0 else float('inf'),
        ((results_batch_4['exact_match'] - results_base['exact_match']) / results_base['exact_match'] * 100) if results_base['exact_match'] > 0 else float('inf'),
        ((results_batch_2['exact_match'] - results_batch_1['exact_match']) / results_batch_1['exact_match'] * 100) if results_batch_1['exact_match'] > 0 else float('inf'),
        ((results_batch_4['exact_match'] - results_batch_1['exact_match']) / results_batch_1['exact_match'] * 100) if results_batch_1['exact_match'] > 0 else float('inf'),
        ((results_batch_4['exact_match'] - results_batch_2['exact_match']) / results_batch_2['exact_match'] * 100) if results_batch_2['exact_match'] > 0 else float('inf')
    ],
    '추론시간_비율': [
        results_batch_1['avg_inference_time'] / results_base['avg_inference_time'],
        results_batch_2['avg_inference_time'] / results_base['avg_inference_time'],
        results_batch_4['avg_inference_time'] / results_base['avg_inference_time'],
        results_batch_2['avg_inference_time'] / results_batch_1['avg_inference_time'],
        results_batch_4['avg_inference_time'] / results_batch_1['avg_inference_time'],
        results_batch_4['avg_inference_time'] / results_batch_2['avg_inference_time']
    ]
})

# CSV 파일 저장 경로 (분석 결과 폴더 내)
csv1_path = os.path.join(analysis_root_dir, 'model_performance_summary_batch_size.csv')
csv2_path = os.path.join(analysis_root_dir, 'detailed_model_comparison_batch_size.csv')
csv3_path = os.path.join(analysis_root_dir, 'batch_size_effect_analysis.csv')

results_summary.to_csv(csv1_path, index=False, encoding='utf-8-sig')
detailed_results.to_csv(csv2_path, index=False, encoding='utf-8-sig')
finetuning_analysis.to_csv(csv3_path, index=False, encoding='utf-8-sig')

print("📁 결과 파일 저장 완료:")
print(f"- {csv1_path}")
print(f"- {csv2_path}")
print(f"- {csv3_path}")

# 시각화 이미지 목록 출력
print("\n📊 생성된 시각화 이미지:")
image_files = os.listdir(image_save_dir)
image_files = [f for f in image_files if f.endswith('.png')]
for i, img_file in enumerate(sorted(image_files), 1):
    print(f"{i}. {os.path.join(image_save_dir, img_file)}")

# 결과 파일들을 압축하여 다운로드 준비
import zipfile
zip_filename = os.path.join(analysis_root_dir, 'HyperCLOVAX_BatchSize_Analysis_Complete_Results.zip')

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    # CSV 파일들 추가
    zipf.write(csv1_path, 'results/model_performance_summary_batch_size.csv')
    zipf.write(csv2_path, 'results/detailed_model_comparison_batch_size.csv')
    zipf.write(csv3_path, 'results/batch_size_effect_analysis.csv')
    
    # 이미지 파일들 추가
    for img_file in image_files:
        img_path = os.path.join(image_save_dir, img_file)
        zipf.write(img_path, f'visualizations/{img_file}')

print(f"\n📦 압축 파일 생성: {zip_filename}")
print("압축 파일 내용:")
print("  📁 results/ - CSV 분석 결과 파일들 (배치 크기 4개 모델 비교)")
print("  📁 visualizations/ - 시각화 이미지들 (배치 크기 4개 모델 비교)")

# Google Colab에서 다운로드 시도
try:
    from google.colab import files
    files.download(zip_filename)
    print("📥 압축 파일 다운로드 완료")
except ImportError:
    print("💾 로컬 환경에서는 다음 경로에 모든 파일이 저장되었습니다:")
    print(f"   분석 결과 폴더: {analysis_root_dir}")
    print(f"   압축 파일: {zip_filename}")
    print(f"   이미지 폴더: {image_save_dir}")

# Google Drive 폴더 구조 안내
print(f"\n📂 Google Drive 폴더 구조:")
print(f"MyDrive/")
print(f"├── HyperCLOVAX_BatchSize_Analysis_Results/")
print(f"│   ├── model_performance_summary_batch_size.csv      # 배치 크기 4개 모델 성능 요약")
print(f"│   ├── detailed_model_comparison_batch_size.csv      # 배치 크기 4개 모델 상세 비교")
print(f"│   ├── batch_size_effect_analysis.csv               # 배치 크기 효과 분석")
print(f"│   ├── HyperCLOVAX_BatchSize_Analysis_Complete_Results.zip")
print(f"│   └── visualization_images/")
print(f"│       ├── 01_batch_size_performance_comparison.png      # 배치 크기 4개 모델 성능 비교")
print(f"│       ├── 02_character_accuracy_distribution.png")
print(f"│       ├── 03_inference_time_comparison.png")
print(f"│       ├── 04_performance_by_text_length.png")
print(f"│       └── 05_accuracy_category_distribution.png")
print(f"├── hyperclova-deobfuscation-lora-with-1-batch-size/")
print(f"├── hyperclova-deobfuscation-lora-with-2-batch-size/")
print(f"├── hyperclova-deobfuscation-lora-with-4-batch-size/")
print(f"└── testdata.csv")

# 결과 요약 출력 (배치 크기 4개 모델)
print("\n=== 최종 성능 요약 (원본 vs 배치 1 vs 2 vs 4) ===")
print(results_summary.round(4))

# 미세조정 효과 요약
best_base_accuracy = results_base['char_accuracy']
best_batch_1_accuracy = results_batch_1['char_accuracy']
best_batch_2_accuracy = results_batch_2['char_accuracy']
best_batch_4_accuracy = results_batch_4['char_accuracy']

print(f"\n=== 미세조정 효과 분석 ===")
print(f"원본 모델 문자 정확도: {best_base_accuracy:.4f}")
print(f"배치 1 모델 문자 정확도: {best_batch_1_accuracy:.4f} ({((best_batch_1_accuracy - best_base_accuracy) / best_base_accuracy * 100):+.2f}%)")
print(f"배치 2 모델 문자 정확도: {best_batch_2_accuracy:.4f} ({((best_batch_2_accuracy - best_base_accuracy) / best_base_accuracy * 100):+.2f}%)")
print(f"배치 4 모델 문자 정확도: {best_batch_4_accuracy:.4f} ({((best_batch_4_accuracy - best_base_accuracy) / best_base_accuracy * 100):+.2f}%)")

# 최고 성능 모델 식별
batch_accuracies = [best_batch_1_accuracy, best_batch_2_accuracy, best_batch_4_accuracy]
best_batch_idx = np.argmax(batch_accuracies)
batch_names = ["배치 1", "배치 2", "배치 4"]
best_model = batch_names[best_batch_idx]
best_accuracy = batch_accuracies[best_batch_idx]

print(f"\n최고 성능 모델: {best_model} 모델 (문자 정확도: {best_accuracy:.4f})")

improvement = ((best_accuracy - best_base_accuracy) / best_base_accuracy * 100)
print(f"원본 모델 대비 성능 향상: {improvement:.2f}%")
print(f"배치 크기 조정 효과: {'매우 효과적' if improvement > 20 else '효과적' if improvement > 10 else '보통' if improvement > 5 else '제한적'}")

# 배치 크기별 성능 트렌드 및 최상의 배치 크기 추천
print(f"\n=== 배치 크기별 성능 트렌드 ===")
bleu_values = [results_batch_1['bleu'], results_batch_2['bleu'], results_batch_4['bleu']]
char_acc_values = [results_batch_1['char_accuracy'], results_batch_2['char_accuracy'], results_batch_4['char_accuracy']]
rouge_values = [results_batch_1['rouge1'], results_batch_2['rouge1'], results_batch_4['rouge1']]

bleu_best_idx = np.argmax(bleu_values)
char_best_idx = np.argmax(char_acc_values)
rouge_best_idx = np.argmax(rouge_values)

print(f"BLEU 점수 최고: {batch_names[bleu_best_idx]} ({bleu_values[bleu_best_idx]:.4f})")
print(f"문자 정확도 최고: {batch_names[char_best_idx]} ({char_acc_values[char_best_idx]:.4f})")
print(f"ROUGE-1 점수 최고: {batch_names[rouge_best_idx]} ({rouge_values[rouge_best_idx]:.4f})")

# 전반적 추천
best_overall = max(set([bleu_best_idx, char_best_idx, rouge_best_idx]), key=[bleu_best_idx, char_best_idx, rouge_best_idx].count)
print(f"\n전반적 추천 배치 크기: {batch_names[best_overall]} (다수 메트릭에서 최고 성능)")